This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import json


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
sample = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?rows=1&api_key='+API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
sample.json()
# it is a nested dictionary under main key 'dataset', then inner keys 'column_names' and 'data'
# will be used in answering further questions

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2019-07-23) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-08-06T22:19:36.310Z',
  'newest_available_date': '2019-08-06',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': 1,
  'transform': None,
  'column_index': None,
  'start_date': '2000-06-07',
  'end_date': '2019-08-06',
  'data': [['2019-08-06',
    93.65,
    95.15,
    93.25,
    93.25,
    None,
    119714.0,
    11218618.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Answers
### 1.Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
# this request will be similar to the sample one, just adding start and end dates to it
afx_json = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)

### 2. Convert the returned JSON object into a Python dictionary.

In [6]:
afx_dict = afx_json.json()
# confirming that obtained element is dictionary
print(type(afx_dict))

<class 'dict'>


### 3. Calculate what the highest and lowest opening prices were for the stock in this period.
    We see (from the sample) that obtained dictionary has the outer key 'dataset' and two inner keys that will help to answer this question:
    - 'column_names' - contains the list of column names
    - 'data' - list of lists for each day entry

In [7]:
# extracting index for the column 'open' - it will be needed for other columns so will define a function
def get_index(dict_name, col_name):
    """ The function returns index of a column name col_name 
        from the list in nested dictionary dict_name"""
    ind = dict_name['dataset']['column_names'].index(col_name)
    return ind
open_ind = get_index(afx_dict, 'Open')

In [8]:
# creating a list of opening prices
open_price = [data[open_ind] for data in afx_dict['dataset']['data']]

# removing None values
open_price = list(filter(None, open_price))

# calculating min and max
min_open_price = min(open_price)
max_open_price = max(open_price)
print('The lowest opening price in 2017 was ' + str(min_open_price))
print('The hightest opening price in 2017 was ' + str(max_open_price))

The lowest opening price in 2017 was 34.0
The hightest opening price in 2017 was 53.11


### 4. What was the largest change in any one day (based on High and Low price)?

In [9]:
# constracting list of differences between high and low prices
high_ind = get_index(afx_dict,'High')
low_ind = get_index(afx_dict,'Low')
daily_diff = [data[high_ind]-data[low_ind] for data in afx_dict['dataset']['data']]

# calculating maximum
max_daily_diff = max(daily_diff)
print('The largest change in one day in 2017 was '+str(max_daily_diff))

The largest change in one day in 2017 was 2.8100000000000023


### 5. What was the largest change between any two days (based on Closing Price)?

In [10]:
# getting data with row-on-row change
day_diff_json = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&transform=diff&api_key='+API_KEY)
day_diff = day_diff_json.json()

In [11]:
# creting list of closing price differences between any two days
close_ind = get_index(day_diff,'Close')
close_diff = []
for row in day_diff['dataset']['data']:
    if row[close_ind] is not None:
        close_diff.append(row[close_ind])
        
# calculating maximum change
max_close_diff = max(close_diff)
print('The largest change between any two days in 2017 was ' + str(max_close_diff))        

The largest change between any two days in 2017 was 1.72


### 6. What was the average daily trading volume during this year?

In [12]:
# requesting time series with cumulative sum transformation 
# will give total sum for the year in the first list in 'data' sub-key
sum_json = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&transform=cumul&api_key='+API_KEY)
sum_dict = sum_json.json()

In [13]:
# for average calculation we will need sum over the year and quantity of entries
vol_ind = get_index(sum_dict,'Traded Volume')
yearly_volume = sum_dict['dataset']['data'][0][vol_ind] 
days_traded = len(sum_dict['dataset']['data'])

# calculating average trading volume
avg_volume = yearly_volume / days_traded
print('The average daily trading volume in 2017 was '+str(avg_volume))

The average daily trading volume in 2017 was 89124.33725490196


### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [14]:
# defining a function for median
def get_median(l_med):
    """The function returns the median of the provided list t"""
    l = sorted(l_med)
    if len(l) % 2 == 1:
        ans = l[int(len(l)/2)+1]
    else:
        ans = (l[int(len(l)/2)]+l[int(len(l)/2)+1])/2
    return ans

In [15]:
# extracting trading volume column from the very first dictionary afx_dict
trading_vol = [row[vol_ind] for row in afx_dict['dataset']['data']]

# calculating median using defined function
trading_median = int(get_median(trading_vol))
print('The median trading volume in 2017 was '+str(trading_median))

The median trading volume in 2017 was 76600
